In [82]:
from keras.preprocessing import image
from keras.utils import np_utils
import numpy as np
import os
import matplotlib.pyplot as plt

In [28]:
# # creating model
# model=ResNet50(weights='imagenet')

102858752/102853048 [==============================] - 1856s 18us/step


## Image Data Pipeline

In [46]:
cwd=os.getcwd()
folders = os.listdir(cwd)
folders

['Image_Data1',
 'FineTuning_FeatureExtraction_ResNet50_Model.ipynb',
 '.ipynb_checkpoints']

In [48]:
image_folders=os.listdir(folders[0])
image_folders

['dogs', 'horses', 'humans', 'cats']

In [197]:
image_data=[]
labels=[]
label_dict={}
i=0

for f in image_folders:
    path=os.path.join(folders[0],f)
    label_dict[f]=i
    i+=1
    for images in os.listdir(path):
        
        img_path=os.path.join(path,images)
        img=image.load_img(img_path,target_size=(224,224))
        img=image.img_to_array(img)
        image_data.append(img)
        labels.append(label_dict[f])
        
image_data=np.array(image_data)
labels=np.array(labels)
print(label_dict)
print(image_data.shape,labels.shape)    

{'dogs': 0, 'horses': 1, 'humans': 2, 'cats': 3}
(808, 224, 224, 3) (808,)


In [198]:
## converting labels to one hot vector
labels=np_utils.to_categorical(labels)
print(labels.shape)

(808, 4)


In [199]:
## shuffling data
import random
data = list(zip(image_data,labels))
random.shuffle(data)
image_data[:],labels[:]= zip(*data)

In [200]:
# from sklearn.model_selection import train_test_split
# train_X,test_X,train_Y,test_Y=train_test_split(image_data,labels,test_size=0.2)
# print(train_X.shape,train_Y.shape)
# print(test_X.shape,test_Y.shape)

## Creating transfer learning model

In [201]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input,decode_predictions
from keras.layers import *
from keras.optimizers import Adam
from keras.models import Model

In [206]:
model=ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

In [207]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
__________________________________________________________________________________________________


In [208]:
## Global Average Pooling Layer
gap = GlobalAveragePooling2D()(model.output)

## Dense Layer 1
d1 = Dense(256,activation='relu')(gap)

## Drop-Out layer for avoiding overfitting
dropout = Dropout(0.5)(d1)

## Final layer to output the probabilities of different classes
fc1 = Dense(4,activation='softmax')(dropout)

In [209]:
new_model=Model(inputs=model.input,outputs=fc1)
new_model.summary()

Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

## Fine Tuning
- By making ResNet50 first 90% layers untrainable

In [192]:
for i in range(len(new_model.layers)):
    print(i,new_model.layers[i])

0 <keras.engine.input_layer.InputLayer object at 0x7f1e3f6402d0>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x7f1e3f6403d0>
2 <keras.layers.convolutional.Conv2D object at 0x7f1e3f6404d0>
3 <keras.layers.normalization.BatchNormalization object at 0x7f1e3f6408d0>
4 <keras.layers.core.Activation object at 0x7f1e3f722c90>
5 <keras.layers.convolutional.ZeroPadding2D object at 0x7f1e3f640510>
6 <keras.layers.pooling.MaxPooling2D object at 0x7f1e3f6779d0>
7 <keras.layers.convolutional.Conv2D object at 0x7f1e3f5e2fd0>
8 <keras.layers.normalization.BatchNormalization object at 0x7f1e3f58d9d0>
9 <keras.layers.core.Activation object at 0x7f1e3f5aeed0>
10 <keras.layers.convolutional.Conv2D object at 0x7f1e3f5aefd0>
11 <keras.layers.normalization.BatchNormalization object at 0x7f1e3f51ce10>
12 <keras.layers.core.Activation object at 0x7f1e3f5bfad0>
13 <keras.layers.convolutional.Conv2D object at 0x7f1e3f4fab90>
14 <keras.layers.convolutional.Conv2D object at 0x7f1e3f3f0610>
15 <keras.lay

In [210]:
## Freezing ResNet50 layers
for i in range(171):
    new_model.layers[i].trainable=False
new_model.summary()

Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

Total params: 24,113,284
Trainable params: 1,580,292
Non-trainable params: 22,532,992
__________________________________________________________________________________________________


In [211]:
## Setting learning Rate to be small so that weights learned by ResNet50 don't get hugely affected during training
adam=Adam(lr=0.000005)
## Compiling Model
new_model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [212]:
## training the model

hist=new_model.fit(image_data,labels,shuffle=True,epochs=10,batch_size=16,validation_split=0.2)

Train on 646 samples, validate on 162 samples
Epoch 1/10
646/646 [==============================] - 144s 223ms/step - loss: 1.5396 - accuracy: 0.3514 - val_loss: 1.1281 - val_accuracy: 0.5432
Epoch 2/10
646/646 [==============================] - 146s 226ms/step - loss: 1.2537 - accuracy: 0.4752 - val_loss: 0.9070 - val_accuracy: 0.6481
Epoch 3/10
646/646 [==============================] - 149s 231ms/step - loss: 1.0958 - accuracy: 0.5743 - val_loss: 0.7589 - val_accuracy: 0.7346
Epoch 4/10
646/646 [==============================] - 149s 230ms/step - loss: 0.9168 - accuracy: 0.6347 - val_loss: 0.6463 - val_accuracy: 0.7963
Epoch 5/10
646/646 [==============================] - 146s 226ms/step - loss: 0.7734 - accuracy: 0.7322 - val_loss: 0.5691 - val_accuracy: 0.8210
Epoch 6/10
646/646 [==============================] - 133s 206ms/step - loss: 0.7606 - accuracy: 0.7260 - val_loss: 0.4972 - val_accuracy: 0.8580
Epoch 7/10
646/646 [==============================] - 106s 164ms/step - loss: 